In [3]:
import json
from graphviz import Digraph
# importOS
import os
import re

class State:
    def __init__(self, label=None):
        self.state_id = None
        self.is_terminating = False
        self.next_states = {}

class NFA:
    def __init__(self):
        self.states = {}
        self.initial_state = None
        self.regex = None

    def disjunction(self,initial_state, r_index, regex, states, last_s_index):
        n_begin_s_index = last_s_index + 1
        n_last_s_index = n_begin_s_index
        n_prev_state = n_begin_s_index
        # r_index from element after '|'
        while r_index < len(regex):
                # print("")
            if regex[r_index] == '(':
                print("(r)")
                # extract the regex in the bracket
                num_of_brackets = 1
                rgx_inside = ""
                l = r_index +1
                while num_of_brackets != 0:
                    if regex[l] == '(':
                        num_of_brackets +=1
                    elif regex[l] == ')':
                        num_of_brackets -=1
                    if num_of_brackets != 0:
                        rgx_inside += regex[l]
                    l+=1
                    # r_index +=1
                print(rgx_inside)
                
                i,last_s_r,f,r_prev_el =self.solve_recursive_grouping(initial_state,r_index,rgx_inside,states,n_last_s_index)
                # new initial state
                new_state = State()
                new_state.state_id = n_begin_s_index 
                new_state.is_terminating = False
                new_state.next_states.update( {'ε404' : r_prev_el} )
                states[n_begin_s_index] = new_state
                
                n_begin_s_index = last_s_r
                n_last_s_index =n_begin_s_index
                print(r_index)
                n_prev_state = r_prev_el
                r_index =  r_index + len(rgx_inside) + 2
            elif regex[r_index] == '|':
                print("or")
                return r_index,n_last_s_index,0
                # call or function
            elif regex[r_index] == '*':
                # print("")
                # new_state = State()
                # new_state.state_id = n_begin_s_index
                # new_state.is_terminating = False
                # new_state.next_states.update( {'e1' : n_begin_s_index +1} )
                # new_state.next_states.update( {'e2' : n_prev_state} )
                # states[n_begin_s_index] = new_state
                # states[n_prev_state].next_states.update( {'e' : n_begin_s_index +1} )
                
                self.zero_or_more(states,n_begin_s_index,n_prev_state)
                
                n_begin_s_index =n_begin_s_index+1
                r_index +=1
                n_last_s_index = n_begin_s_index

            elif regex[r_index] == '+':
                # new_state = State()
                # new_state.state_id = n_begin_s_index
                # new_state.is_terminating = False
                # new_state.next_states.update( {'e1' : n_begin_s_index +1} )
                # new_state.next_states.update( {'e2' : n_prev_state} )
                # states[n_begin_s_index] = new_state
                # # states[n_prev_state].next_states.update( {'e' : n_begin_s_index +1} )
                
                self.one_or_more(states,n_begin_s_index,n_prev_state)
                
                n_begin_s_index =n_begin_s_index+1
                r_index +=1
                n_last_s_index = n_begin_s_index
            elif regex[r_index] == '?':
                # print("")
                # new_state = State()
                # new_state.state_id = n_begin_s_index
                # new_state.is_terminating = False
                # new_state.next_states.update( {'e1' : n_begin_s_index +1} )
                # # new_state.next_states.update( {'e2' : n_prev_state} )
                # states[n_begin_s_index] = new_state
                # states[n_prev_state].next_states.update( {'e' : n_begin_s_index +1} )
                
                self.zero_or_one(states,n_begin_s_index,n_prev_state)
                n_begin_s_index =n_begin_s_index+1
                r_index +=1
                n_last_s_index = n_begin_s_index
            elif regex[r_index] == '[':
                print("[")
                # extract the regex in the bracket
                # num_of_brackets = 1
                # rgx_inside = ""
                # l = r_index +1
                # while num_of_brackets != 0:
                #     if regex[l] == '[':
                #         num_of_brackets +=1
                #     elif regex[l] == ']':
                #         num_of_brackets -=1
                #     if num_of_brackets != 0:
                #         rgx_inside += regex[l]
                #     l+=1
                #     # r_index +=1
                # print(rgx_inside)
                # # new initial state
                # new_state = State()
                # new_state.state_id = n_begin_s_index 
                # new_state.is_terminating = False
                # new_state.next_states.update( {'['+rgx_inside +']' : n_begin_s_index +1} )
                # states[n_begin_s_index] = new_state
                self.ranges(regex,r_index,n_begin_s_index,states)
                
                n_prev_state = n_begin_s_index
                n_begin_s_index =n_begin_s_index+1
                r_index =  r_index + len(rgx_inside) + 2
                n_last_s_index = n_begin_s_index
            else:
                new_state = State()
                new_state.state_id = n_begin_s_index
                new_state.is_terminating = False
                # 
                if (regex[r_index] == '.'):
                    new_state.next_states.update( {'ε010' : n_begin_s_index +1} )
                else:
                    new_state.next_states.update( {regex[r_index] : n_begin_s_index +1} )
                states[n_begin_s_index] = new_state
                n_prev_state = n_begin_s_index
                n_begin_s_index =n_begin_s_index+1
                r_index +=1
                n_last_s_index = n_begin_s_index
                
        finish =1
        return -1,n_last_s_index,finish

    def save_json(self,states,initial_state):
        state_data = {
        "startingState": "S" + str(initial_state),
        # "states": {}
        }

        for state_id, state_obj in states.items():
            state_info = {
                # "state_id": state_obj.state_id,
                "isTerminatingState": state_obj.is_terminating,
                # "Transitions": {}
            }

            if not state_obj.is_terminating:
                for transition, next_state_id in state_obj.next_states.items():
                    state_info[transition] = "S" + str(next_state_id)
            # else:
                # remove the transitions key if the state is terminating
                # state_info.pop("Transitions")

            state_data["S" + str(state_obj.state_id)] = state_info

        # Writing to JSON file
        with open("states.json", "w") as json_file:
            json.dump(state_data, json_file, indent=4)
    def solve_recursive_grouping(self,initial_state, r_index, regex, states, last_s_index):
        new_state2 = State()
        new_state2.state_id = last_s_index+1
        new_state2.is_terminating = False
        states[last_s_index+1] = new_state2
        # last_s_index -> end_state
        
        n_begin_s_index = last_s_index + 1 #bstart
        n_last_s_index = n_begin_s_index # bend
        n_prev_state = n_begin_s_index # b_ch
        n_prev_state_r = n_begin_s_index # b_prev
        r_index =0
        while r_index < len(regex):
            
            if regex[r_index] == '(':
                print("(r)")
                # extract the regex in the bracket
                num_of_brackets = 1
                rgx_inside = ""
                l = r_index +1
                while num_of_brackets != 0:
                    if regex[l] == '(':
                        num_of_brackets +=1
                    elif regex[l] == ')':
                        num_of_brackets -=1
                    if num_of_brackets != 0:
                        rgx_inside += regex[l]
                    l+=1
                    # r_index +=1
                print(rgx_inside)
                
                # call thompsons_construction function
                # solve_recursive_grouping(rgx_inside)
                # last_s_r -> end
                # r_prev_el -> prev 
                i,last_s_r,f,r_prev_el =self.solve_recursive_grouping(initial_state,r_index,rgx_inside,states,n_last_s_index)
                # new initial state
                new_state = State()
                new_state.state_id = n_begin_s_index 
                new_state.is_terminating = False
                new_state.next_states.update( {'ε654' : r_prev_el} )
                states[n_begin_s_index] = new_state
                
                n_begin_s_index = last_s_r
                n_last_s_index =n_begin_s_index
                print(r_index)
                n_prev_state = r_prev_el
                r_index =  r_index + len(rgx_inside) + 2
                
            elif regex[r_index] == '|':
                print("or")
                # return r_index,n_last_s_index,0
                begin_disjunction = n_last_s_index + 1
                
                r_index,d_last_s_index, finish = self.disjunction(initial_state, r_index+1, regex, states, n_last_s_index)
                
                print("orr",begin_disjunction,d_last_s_index)
                # new initial state
                new_state = State()
                new_state.state_id = d_last_s_index + 1
                new_state.is_terminating = False
                new_state.next_states.update( {'ε144' : n_prev_state_r} )
                new_state.next_states.update( {'ε2775' : begin_disjunction  } )
                states[d_last_s_index + 1] = new_state
                #new end state
                new_state = State()
                new_state.state_id = d_last_s_index + 2
                new_state.is_terminating = False
                states[d_last_s_index + 2] = new_state
                
                # new 
                new_state = State()
                new_state.state_id = n_last_s_index
                new_state.is_terminating = False
                new_state.next_states.update( {'ε456' : d_last_s_index + 2} )
                states[n_last_s_index] = new_state

                # new
                # states[d_last_s_index].next_states.update( {'e' : d_last_s_index + 2} )
                new_state = State()
                new_state.state_id = d_last_s_index
                new_state.is_terminating = False
                new_state.next_states.update( {'ε325' : d_last_s_index + 2} )
                states[d_last_s_index] = new_state
                
                n_last_s_index = d_last_s_index + 2
                # initial_state = d_last_s_index + 1
                n_prev_state = d_last_s_index + 1
                n_begin_s_index = d_last_s_index + 2
                n_prev_state_r = d_last_s_index + 1
                
                
                
                if finish == 1:
                    r_index = len(regex)
                # call or function
            elif regex[r_index] == '*':
                # print("")
                # new_state = State()
                # new_state.state_id = n_begin_s_index
                # new_state.is_terminating = False
                # new_state.next_states.update( {'ε5' : n_begin_s_index +1} )
                # new_state.next_states.update( {'ε6' : n_prev_state} )
                # states[n_begin_s_index] = new_state
                # states[n_prev_state].next_states.update( {'ε100' : n_begin_s_index +1} )
                
                self.zero_or_more(states,n_begin_s_index,n_prev_state)
                
                n_begin_s_index =n_begin_s_index+1
                r_index +=1
                n_last_s_index = n_begin_s_index
            elif regex[r_index] == '+':
                # new_state = State()
                # new_state.state_id = n_begin_s_index
                # new_state.is_terminating = False
                # new_state.next_states.update( {'e1' : n_begin_s_index +1} )
                # new_state.next_states.update( {'e2' : n_prev_state} )
                # states[n_begin_s_index] = new_state
                # states[n_prev_state].next_states.update( {'e' : n_begin_s_index +1} )
                self.one_or_more(states,n_begin_s_index,n_prev_state)
                n_begin_s_index =n_begin_s_index+1
                r_index +=1
                n_last_s_index = n_begin_s_index
            elif regex[r_index] == '?':
                # print("")
                # new_state = State()
                # new_state.state_id = n_begin_s_index
                # new_state.is_terminating = False
                # new_state.next_states.update( {'e1' : n_begin_s_index +1} )
                # # new_state.next_states.update( {'e2' : n_prev_state} )
                # states[n_begin_s_index] = new_state
                # states[n_prev_state].next_states.update( {'e' : n_begin_s_index +1} )
                
                self.zero_or_one(states,n_begin_s_index,n_prev_state)
                
                n_begin_s_index =n_begin_s_index+1
                r_index +=1
                n_last_s_index = n_begin_s_index
            elif regex[r_index] == '[':
                print("[")
                # extract the regex in the bracket
                # num_of_brackets = 1
                # rgx_inside = ""
                # l = r_index +1
                # while num_of_brackets != 0:
                #     if regex[l] == '[':
                #         num_of_brackets +=1
                #     elif regex[l] == ']':
                #         num_of_brackets -=1
                #     if num_of_brackets != 0:
                #         rgx_inside += regex[l]
                #     l+=1
                #     # r_index +=1
                # print(rgx_inside)
                # # new initial state
                # new_state = State()
                # new_state.state_id = n_begin_s_index 
                # new_state.is_terminating = False
                # new_state.next_states.update( {'['+rgx_inside +']' : n_begin_s_index +1} )
                # states[n_begin_s_index] = new_state
                
                rgx_inside = self.ranges(regex,r_index,n_begin_s_index,states)
                
                n_prev_state = n_begin_s_index
                n_begin_s_index =n_begin_s_index+1
                r_index =  r_index + len(rgx_inside) + 2
                n_last_s_index = n_begin_s_index
            else:
                # print("ppp",regex[r_index],n_begin_s_index,regex[r_index +1])
                new_state = State()
                new_state.state_id = n_begin_s_index
                new_state.is_terminating = False
                # 
                if (regex[r_index] == '.'):
                    new_state.next_states.update( {'ε4578' : n_begin_s_index +1} )
                else :
                    new_state.next_states.update( {regex[r_index] : n_begin_s_index +1} )
                states[n_begin_s_index] = new_state
                n_prev_state = n_begin_s_index
                n_begin_s_index =n_begin_s_index+1
                r_index +=1
                n_last_s_index = n_begin_s_index
                
        finish =1
        return -1,n_last_s_index,finish,n_prev_state_r
    def ranges(self,regex,r_index,begin_s_index,states):
        num_of_brackets = 1
        rgx_inside = ""
        l = r_index +1
        while num_of_brackets != 0:
            if regex[l] == '[':
                num_of_brackets +=1
            elif regex[l] == ']':
                num_of_brackets -=1
            if num_of_brackets != 0:
                rgx_inside += regex[l]
            l+=1
            # r_index +=1
        print(rgx_inside)
        # new initial state
        new_state = State()
        new_state.state_id = begin_s_index 
        new_state.is_terminating = False
        new_state.next_states.update( {'['+rgx_inside +']' : begin_s_index +1} )
        states[begin_s_index] = new_state
        return rgx_inside
    def zero_or_one(self,states,begin_s_index,prev_state):
        new_state = State()
        new_state.state_id = begin_s_index
        new_state.is_terminating = False
        new_state.next_states.update( {'ε41' : begin_s_index +1} )
        # in case of more will go back to the prev state
        # new_state.next_states.update( {'e2' : prev_state} )
        states[begin_s_index] = new_state
        # for zero will go to next state with e moves
        states[prev_state].next_states.update( {'ε33' : begin_s_index +1} )
    def one_or_more(self,states,begin_s_index,prev_state):
        # print("")
        # print("")
        # zero or more 
        new_state = State()
        new_state.state_id = begin_s_index
        new_state.is_terminating = False
        new_state.next_states.update( {'ε15' : begin_s_index +1} )
        # in case of more will go back to the prev state
        new_state.next_states.update( {'ε27' : prev_state} )
        states[begin_s_index] = new_state
        # for zero will go to next state with e moves
        # states[prev_state].next_states.update( {'e' : begin_s_index +1} )
    def zero_or_more(self,states,begin_s_index,prev_state):
        new_state = State()
        new_state.state_id = begin_s_index
        new_state.is_terminating = False
        new_state.next_states.update( {'ε10' : begin_s_index +1} )
        # in case of more will go back to the prev state
        new_state.next_states.update( {'ε20' : prev_state} )
        states[begin_s_index] = new_state
        # for zero will go to next state with e moves
        states[prev_state].next_states.update( {'ε100' : begin_s_index +1} )
    def thompsons_construction(self,regex):
        r_index = 0
        states = {}
        s_0 = State()
        s_0.state_id =0
        s_0.is_terminating =False
        states.update({0:s_0})
        # point to the first state
        begin_s_index = 0  #start_state 
        # point to the last state
        last_s_index = 0 #end_state
        # point to the initial state (starting state)
        initial_state = 0 #prev_start
        # previous element that will used in case of repetition
        prev_state = 0 #prev_char
        while r_index < len(regex):
            if regex[r_index] == '(':
                print("(n)")
                # extract rgx in the bracket
                # begin_grouping = last_s_index + 1
                # print(regex[begin_grouping],"s" + str(begin_grouping))
                # extract the regex in the bracket
                num_of_brackets = 1
                rgx_inside = ""
                l = r_index +1
                while num_of_brackets != 0:
                    if regex[l] == '(':
                        num_of_brackets +=1
                    elif regex[l] == ')':
                        num_of_brackets -=1
                    if num_of_brackets != 0:
                        rgx_inside += regex[l]
                    l+=1
                    # r_index +=1
                print(rgx_inside)
                
                # call thompsons_construction function
                i,n_last_s_index_r,f,n_prev_state_r = self.solve_recursive_grouping(initial_state,r_index,rgx_inside,states,last_s_index)
                # new initial state
                new_state = State()
                new_state.state_id = last_s_index 
                new_state.is_terminating = False
                new_state.next_states.update( {'ε7' : n_prev_state_r} )
                states[last_s_index] = new_state
                
                last_s_index = n_last_s_index_r
                begin_s_index = last_s_index
                r_index =  r_index + len(rgx_inside) + 2
                print(r_index)
                prev_state = n_prev_state_r
                
            elif regex[r_index] == '|':
                print("or")
                # call or function
                begin_disjunction = last_s_index + 1
                r_index,d_last_s_index, finish = self.disjunction(initial_state, r_index+1, regex, states, last_s_index)
                # new initial state
                new_state = State()
                new_state.state_id = d_last_s_index + 1
                new_state.is_terminating = False
                new_state.next_states.update( {'ε144' : begin_disjunction} )
                new_state.next_states.update( {'ε277' : initial_state  } )
                states[d_last_s_index + 1] = new_state
                #new end state
                new_state = State()
                new_state.state_id = d_last_s_index + 2
                new_state.is_terminating = False
                states[d_last_s_index + 2] = new_state
                
                # new 
                new_state = State()
                new_state.state_id = last_s_index
                new_state.is_terminating = False
                new_state.next_states.update( {'ε789' : d_last_s_index + 2} )
                states[last_s_index] = new_state

                # new
                # states[d_last_s_index].next_states.update( {'e' : d_last_s_index + 2} )
                new_state = State()
                new_state.state_id = d_last_s_index
                new_state.is_terminating = False
                new_state.next_states.update( {'ε684' : d_last_s_index + 2} )
                states[d_last_s_index] = new_state
                
                last_s_index = d_last_s_index + 2
                initial_state = d_last_s_index + 1
                prev_state = d_last_s_index + 1
                begin_s_index = d_last_s_index + 2
                
                
                
                if finish == 1:
                    r_index = len(regex)
            elif regex[r_index] == '*':
                # print("")
                # zero or more 
                
                # new_state = State()
                # new_state.state_id = begin_s_index
                # new_state.is_terminating = False
                # new_state.next_states.update( {'e1' : begin_s_index +1} )
                # # in case of more will go back to the prev state
                # new_state.next_states.update( {'e2' : prev_state} )
                # states[begin_s_index] = new_state
                # # for zero will go to next state with e moves
                # states[prev_state].next_states.update( {'e' : begin_s_index +1} )
                self.zero_or_more(states,begin_s_index,prev_state)
                begin_s_index =begin_s_index+1
                r_index +=1
                last_s_index = begin_s_index
            
            elif regex[r_index] == '+':
                # print("")
                # print("")
                # zero or more 
                
                # new_state = State()
                # new_state.state_id = begin_s_index
                # new_state.is_terminating = False
                # new_state.next_states.update( {'e1' : begin_s_index +1} )
                # # in case of more will go back to the prev state
                # new_state.next_states.update( {'e2' : prev_state} )
                # states[begin_s_index] = new_state
                # # for zero will go to next state with e moves
                # # states[prev_state].next_states.update( {'e' : begin_s_index +1} )
                self.one_or_more(states,begin_s_index,prev_state)
                
                begin_s_index =begin_s_index+1
                r_index +=1
                last_s_index = begin_s_index
                
            elif regex[r_index] == '?':
                # print("")
                # print("")
                # zero or more 
                # new_state = State()
                # new_state.state_id = begin_s_index
                # new_state.is_terminating = False
                # new_state.next_states.update( {'e1' : begin_s_index +1} )
                # # in case of more will go back to the prev state
                # # new_state.next_states.update( {'e2' : prev_state} )
                # states[begin_s_index] = new_state
                # # for zero will go to next state with e moves
                # states[prev_state].next_states.update( {'e' : begin_s_index +1} )
                self.zero_or_one(states,begin_s_index,prev_state)
                begin_s_index =begin_s_index+1
                r_index +=1
                last_s_index = begin_s_index
            
            elif regex[r_index] == '[':
                
                rgx_inside = self.ranges(regex,r_index,begin_s_index,states)
                # update the indices
                prev_state = begin_s_index
                begin_s_index =begin_s_index+1
                r_index =  r_index + len(rgx_inside) + 2
                last_s_index = begin_s_index
            else:
                new_state = State()
                new_state.state_id = begin_s_index
                new_state.is_terminating = False
                if (regex[r_index] == '.'):
                    new_state.next_states.update( {'ε78' : begin_s_index +1} )
                else:
                    new_state.next_states.update( {regex[r_index] : begin_s_index +1} )
                states[begin_s_index] = new_state
                
                prev_state = begin_s_index
                begin_s_index =begin_s_index+1
                r_index +=1
                last_s_index = begin_s_index
        # print loop and print the states
        # print each state and its transitions like json
        
        new_state = State()
        new_state.state_id = last_s_index
        new_state.is_terminating = True
        states[last_s_index] = new_state


        # self.print_states(states,initial_state)
        self.save_json(states,initial_state)
        
        
        return 0
    def print_states(self,states,initial_state):
        print("{ \"startingState\": \"S"+str(initial_state)+"\",")
        #  sort states according to their state_id
        
        states = dict(sorted(states.items()))
        for state in states:
            print("  S"+str(states[state].state_id) + ": {")
            print("isTerminatingState: " + str(states[state].is_terminating) + ",")
            if states[state].is_terminating:
                print("}")
            else:
                print("Transitions: {")
                for transition in states[state].next_states:
                    print(transition + ": " + "S" + str(states[state].next_states[transition]) + ",")
                print("}")
                print("}")
        print("}")
    def draw_graph2(self):
        # "C:\Program Files\Graphviz\bin\dot.exe" run this command in the terminal to install graphviz
        # ! "C:\Program Files\Graphviz\bin\dot.exe"
        os.environ["PATH"] += os.pathsep + 'C:/Program Files/Graphviz/bin/'
        # Load state machine definition from JSON file
        with open("states.json", "r") as json_file:
            state_machine_definition = json.load(json_file)
        #  want to set direction to left to right


        # Initialize the Digraph object
        dot = Digraph()

        # Set the direction of the graph
        dot.attr(rankdir="LR")

        # Add states to the graph
        for state_id, state_info in state_machine_definition["states"].items():
            label = f"{state_id}"
            if state_info['isTerminatingState']:
                dot.node(state_id, label=label, shape="doublecircle")
            else:
                dot.node(state_id, label=label)

        # Add transitions to the graph
        for state_id, state_info in state_machine_definition["states"].items():
            for event, next_state_id in state_info["Transitions"].items():
                dot.edge(state_id, next_state_id, label=event)

        # Save the graph to a file
        dot.render("state_machine_graph", format="png", cleanup=True)
    def draw_graph(self):

        # Add Graphviz bin directory to PATH
        os.environ["PATH"] += os.pathsep + 'C:/Program Files/Graphviz/bin/'

        # Load state machine definition from JSON file
        with open("states.json", "r") as json_file:
            state_machine_definition = json.load(json_file)

        # Initialize the Digraph object
        dot = Digraph()

        # Set the direction of the graph (left to right)
        dot.attr(rankdir="LR")

        # Add states to the graph
        for state_id, state_info in state_machine_definition.items():
            label = f"{state_id}"
            if state_id == state_machine_definition["startingState"]:
                # dot.node(state_id, label=label, shape="doublecircle")
                print("")
            elif isinstance(state_info, dict) and not state_info.get('isTerminatingState', False):
                dot.node(state_id, label=label)
            elif isinstance(state_info, dict):
                dot.node(state_id, label=label, shape="doublecircle")

        # Add transitions to the graph
        for state_id, state_info in state_machine_definition.items():
            if state_id != "startingState" and isinstance(state_info, dict):
                for action, next_state_id in state_info.items():
                    if action != "isTerminatingState":
                        dot.edge(state_id, next_state_id, label=action)

        # Add arrow to the starting state
        dot.attr('edge',dir="forward")
        dot.edge('', state_machine_definition["startingState"], label='')

        # Save the graph to a file
        dot.render("state_machine_graph", format="png", cleanup=True)
    def validate_rounded_braces(self,string):
        stack = []
        for char in string:
            if char == '(':
                stack.append(char)
            elif char == ')':
                if not stack:
                    return False  # There's a closing brace with no matching opening brace
                stack.pop()
        return not stack  # If the stack is empty, braces are balanced
    def validate_square_braces(self,string):
        stack = []
        for char in string:
            if char == '[':
                stack.append(char)
            elif char == ']':
                if not stack:
                    return False  # There's a closing brace with no matching opening brace
                stack.pop()
        return not stack
    def validate_regex(self,regex):
        disallowed_pattern = r'[\{\}]'  # Pattern to disallow
        if re.search(disallowed_pattern, regex):
            return False
        allowed_chars_pattern = r'[\(\)a-zA-Z\[\]*?|+.0-9-]'
        return bool(re.match(allowed_chars_pattern, regex)) and self.validate_rounded_braces(regex) and self.validate_square_braces(regex)
        
        
    def expand_range(self,pattern):
        expanded = []
        expanded_pattern = ''
        i = 0
        while i < len(pattern):
            if pattern[i] == '[':
                expanded_pattern += '('
                i += 1
                while pattern[i] != ']':
                    if pattern[i + 1] == '-' and i + 2 < len(pattern) and ord(pattern[i]) < ord(pattern[i + 2]):
                        expanded.extend(chr(c) for c in range(ord(pattern[i]), ord(pattern[i + 2]) + 1))
                        i += 2
                    else:
                        expanded.append(pattern[i])
                    i += 1
                expanded_pattern += '|'.join(expanded)
                expanded_pattern += ')'
                expanded = []
            else:
                expanded_pattern += pattern[i]
            i += 1
        return expanded_pattern

def main():
    regex = input("Enter a regular expression: ")
    # trim space
    regex = regex.replace(" ", "")
    nfa = NFA()
    if nfa.validate_regex(regex):
        print("Valid regex")
        rr = nfa.expand_range(regex)
        # print(rr)
        nfa.thompsons_construction(rr)
        nfa.draw_graph()
    else :
        print("Invalid regex")
        print("Please enter a valid regex")


if __name__ == "__main__":
    main()
#((A)((A)*))


Valid regex



In [4]:
## convert from NFA to DFA
import json
from collections import deque
from graphviz import Digraph
import os

os.environ["PATH"] += os.pathsep + 'C:/Program Files/Graphviz/bin/'

class State:
    def __init__(self, label=None):
        self.state_id = None
        self.is_terminating = False
        self.next_in_states = {}

class DFA:
    def __init__(self):
        self.in_states = {}
        self.initial_state = None
        self.regex = None

    def read_json(self, file_path):
        with open(file_path, "r") as json_file:
            state_machine_definition = json.load(json_file)
        
        # Extract in_states and initial state from the JSON data
        self.in_states = state_machine_definition  # assuming JSON structure is already in expected format
        self.initial_state = state_machine_definition["startingState"]
        # remove the initial state from the in_states
        self.in_states.pop("startingState")

    def epsilon_closure(self, states):
        epsilon_closure_set = set(states)
        stack = list(states)
        while stack:
            current_state = stack.pop()
            listt = []
            for transition in self.in_states[current_state]:
                if transition[0] == 'ε':
                    listt.append(self.in_states[current_state][transition])
            for next_state in listt:
                if next_state != []:
                    if next_state not in epsilon_closure_set:
                        epsilon_closure_set.add(next_state)
                        stack.append(next_state)
        return epsilon_closure_set

    def get_all_actions(self):
        actions = set()
        for state in self.in_states.values():
            for transition in state:
                if transition[0] not in ['ε', 'ε1', 'ε2', 'isTerminatingState']:
                    actions.update(transition)
        actions.discard('ε')  # Remove epsilon from actions
        return actions

    def get_terminating_state(self):
        terminating_states = []
        for state in self.in_states.keys():
            if self.in_states[state]["isTerminatingState"] == True:
                terminating_states.append(state)
        return terminating_states

    def check_terminating_state(self, states):
        terminating_states = self.get_terminating_state()
        for state in states:
            if state in terminating_states:
                return True
        return False

    def nfa2dfa(self):
        # Initialize the DFA states dictionary
        dfa_states = {}
        initial_epsilon_closure = self.epsilon_closure([self.initial_state])
        dfa_states["startingState"] = ' '.join(initial_epsilon_closure)
        dfa_states[' '.join(initial_epsilon_closure)] = {'isTerminatingState': False}
        # Create a queue to process the states
        state_queue = deque([tuple(initial_epsilon_closure)])

        # While there are unprocessed states in the queue
        while state_queue:
            current_state = state_queue.popleft()

            # Get all possible symbols for the NFA
            symbols = self.get_all_actions()

            # Initialize the transition dictionary for the current DFA state
            dfa_states[' '.join(current_state)]['isTerminatingState'] = False
            if self.check_terminating_state(current_state):
                dfa_states[' '.join(current_state)]["isTerminatingState"] = True

            # For each symbol, find the epsilon closure of the next state
            for symbol in symbols:
                next_states = []
                for nfa_state_label in current_state:
                    nfa_state = self.in_states[nfa_state_label]
                    transitions = []
                    transitions.append(nfa_state.get(symbol, ""))
                    if transitions[0] != "":
                        for transition in transitions:
                            transition = [transition] if isinstance(transition, str) else transition
                            next_states.extend(self.epsilon_closure(transition))
                if next_states == []:
                    continue
                next_states = list(set(next_states))  # Remove duplicates
                next_states.sort()  # Sort for consistency

                # Add the next state to the DFA if it's not already in there
                if ' '.join(next_states) not in dfa_states:
                    dfa_states[' '.join(next_states)] = {'isTerminatingState': False}
                    state_queue.append(next_states)

                # Add the transition from the current DFA state to the next DFA state
                dfa_states[' '.join(current_state)][symbol] = ' '.join(next_states)

        return dfa_states

    def visualize(self, filename='dfa_graph'):
        dfa_states = self.nfa2dfa()
        dot = Digraph()
        for state, transitions in dfa_states.items():
            if state != "startingState":
                state_label = state
                if transitions['isTerminatingState']:
                    dot.node(state_label, state_label, shape='doublecircle')
                else:
                    dot.node(state_label, state_label)
                for symbol, next_state in transitions.items():
                    if symbol == 'isTerminatingState':
                        continue
                    dot.edge(state_label, next_state, label=symbol)
        dot.render(filename, format='png', cleanup=True)

    def save_json(self, filename='dfa_states.json'):
        dfa_states = self.nfa2dfa()
        dfa_states_json = {}
        for state, transitions in dfa_states.items():
            if state != "startingState":
                dfa_states_json[state] = transitions

        with open(filename, 'w') as json_file:
            json.dump(dfa_states_json, json_file, indent=4)

nfa = DFA()
nfa.read_json("states.json")
print(nfa.in_states)
print(nfa.initial_state)
print("get_all_actions",nfa.get_all_actions())

# print(nfa.epsilon_closure(["S4"]))

print("fff",nfa.nfa2dfa())
nfa.visualize()
nfa.save_json()


{'S0': {'isTerminatingState': False, 'a': 'S1'}, 'S1': {'isTerminatingState': False, 'a': 'S2'}, 'S2': {'isTerminatingState': False, 'a': 'S3'}, 'S3': {'isTerminatingState': False, 'b': 'S4'}, 'S4': {'isTerminatingState': False, 'b': 'S5'}, 'S5': {'isTerminatingState': True}}
S0
get_all_actions {'a', 'b'}
states ('S0',)
terminating_states ['S5']
states ('S1',)
terminating_states ['S5']
states ('S2',)
terminating_states ['S5']
states ('S3',)
terminating_states ['S5']
states ('S4',)
terminating_states ['S5']
states ('S5',)
terminating_states ['S5']
fff {'startingState': ('S0',), ('S0',): {'isTerminatingState': False, 'a': ['S1']}, ('S1',): {'isTerminatingState': False, 'a': ['S2']}, ('S2',): {'isTerminatingState': False, 'a': ['S3']}, ('S3',): {'isTerminatingState': False, 'b': ['S4']}, ('S4',): {'isTerminatingState': False, 'b': ['S5']}, ('S5',): {'isTerminatingState': True}}
states ('S0',)
terminating_states ['S5']
states ('S1',)
terminating_states ['S5']
states ('S2',)
terminating_sta

In [8]:
## convert from DFA to MinDFA
class MinimizeDFA:
    def __init__(self, dfa_states):
        self.dfa_states = dfa_states

    def minimize_dfa(self):
        # Step 1: Split states into two sets - accepting and non-accepting
        
        accepting_states = []
        non_accepting_states = []
        print("self.dfa_states",self.dfa_states)
        for state, transitions in self.dfa_states.items():
            print("transitions",transitions)
            print("state",state)
            print("step 1 start----------------------------------")

            if isinstance(transitions, dict) and 'isTerminatingState' in transitions:
                if transitions['isTerminatingState']:
                    accepting_states.append(state)
                else:
                    non_accepting_states.append(state)
            # is_terminating_state = transitions.get('isTerminatingState', False)  # Extracting the terminating state information directly
            # if is_terminating_state:
            #     accepting_states.append(state)
            # else:
            #     non_accepting_states.append(state)
        print("step 1 end -------------------------------------------")
        print("accepting_states",accepting_states)
        print("non_accepting_states",non_accepting_states)
        # Step 2: Initialize partition
        print("step 2 start----------------------------------")
        partition = [accepting_states, non_accepting_states]
        new_partition = []
        print("step 2 end -------------------------------------------")
        # Step 3: Refinement loop
        while new_partition != partition:
            print("step 3 start----------------------------------")
            partition = new_partition if new_partition else partition
            new_partition = []

            for group in partition:
                if len(group) <= 1:
                    new_partition.append(group)
                    continue

                # Partition states based on transitions
                transitions_partition = {}
                for state in group:
                    # Omitting the 'isTerminatingState' key when constructing the transitions dictionary
                    transitions = {symbol: next_state for symbol, next_state in self.dfa_states[state].items() if symbol != 'isTerminatingState'}
                    transitions_str = str(transitions)
                    if transitions_str not in transitions_partition:
                        transitions_partition[transitions_str] = [state]
                    else:
                        transitions_partition[transitions_str].append(state)

                new_partition.extend(list(transitions_partition.values()))
        print("step 3 end----------------------------------")
        # Step 4: Construct minimized DFA
        minimized_dfa_states = {}
        initial_state = None
        for i, group in enumerate(new_partition):
            print("step 4 start----------------------------------")
            state_label = f'Q{i}'
            
            if self.dfa_states['startingState'] in group:
                initial_state = state_label
            for state in group:
                minimized_dfa_states[state] = state_label
        print("step 4 end----------------------------------")
        # Step 5: Update transitions
        print("minimized_dfa_states",minimized_dfa_states)
        print("initial_state",initial_state)
        minimized_dfa = {'startingState': initial_state}
        for state, transitions in self.dfa_states.items():
            print("step 5 start----------------------------------")
            if state == 'startingState':
                continue
            minimized_state = minimized_dfa_states[state]
            minimized_transitions = {}
            for symbol, next_state in transitions.items():
                if symbol == 'isTerminatingState':
                    minimized_transitions[symbol] = next_state
                else:
                    # Ensure next_state is a single state label
                    if isinstance(next_state, list):
                        next_state_labels = [minimized_dfa_states[state] for state in next_state]
                        minimized_transitions[symbol] = next_state_labels
                    else:
                        minimized_transitions[symbol] = minimized_dfa_states[next_state]
            minimized_dfa[minimized_state] = minimized_transitions
        print("step 5 end----------------------------------")
        return minimized_dfa
    

    # def visualize(self, name='output/min.gv', view=False):
    #     graph = graphviz.Digraph(engine='dot')
    #     for state, transitions in self.dfa_states.items(): 
    #         if state == 'startingState':
    #             continue
    #         if transitions['isTerminatingState']:
    #             graph.node(state, shape='doublecircle')
    #         else:
    #             graph.node(state, shape='circle')
    #         for char, next_state in transitions.items():
    #             if char == 'isTerminatingState':
    #                 continue
    #             children_states = next_state.split(',')
    #             for child in children_states:
    #                 graph.edge(state, child, label=char)
    #     graph.render("output/min", view=False)


# Usage example:
# Convert from NFA to DFA


# Convert DFA to Minimized DFA
dfa_states = nfa.nfa2dfa()
minimizer = MinimizeDFA(dfa_states)
minimized_dfa = minimizer.minimize_dfa()




states ('S0',)
terminating_states ['S5']
states ('S1',)
terminating_states ['S5']
states ('S2',)
terminating_states ['S5']
states ('S3',)
terminating_states ['S5']
states ('S4',)
terminating_states ['S5']
states ('S5',)
terminating_states ['S5']
self.dfa_states {'startingState': ('S0',), ('S0',): {'isTerminatingState': False, 'a': ['S1']}, ('S1',): {'isTerminatingState': False, 'a': ['S2']}, ('S2',): {'isTerminatingState': False, 'a': ['S3']}, ('S3',): {'isTerminatingState': False, 'b': ['S4']}, ('S4',): {'isTerminatingState': False, 'b': ['S5']}, ('S5',): {'isTerminatingState': True}}
transitions ('S0',)
state startingState
step 1 start----------------------------------
transitions {'isTerminatingState': False, 'a': ['S1']}
state ('S0',)
step 1 start----------------------------------
transitions {'isTerminatingState': False, 'a': ['S2']}
state ('S1',)
step 1 start----------------------------------
transitions {'isTerminatingState': False, 'a': ['S3']}
state ('S2',)
step 1 start-------

KeyError: 'S1'

In [9]:
## convert from DFA to MinDFA
class MinimizeDFA:
    def __init__(self, dfa_states):
        self.dfa_states = dfa_states

    def minimize_dfa(self):
        # Step 1: Split states into two sets - accepting and non-accepting
        accepting_states = []
        non_accepting_states = []
        for state, transitions in self.dfa_states.items():
            is_terminating_state = transitions.get('isTerminatingState', False)  # Extracting the terminating state information directly
            if is_terminating_state:
                accepting_states.append(state)
            else:
                non_accepting_states.append(state)

        # Step 2: Initialize partition
        partition = [accepting_states, non_accepting_states]
        new_partition = []
        
        # Step 3: Refinement loop
        while new_partition != partition:
            partition = new_partition if new_partition else partition
            new_partition = []

            for group in partition:
                if len(group) <= 1:
                    new_partition.append(group)
                    continue

                # Partition states based on transitions
                transitions_partition = {}
                for state in group:
                    # Omitting the 'isTerminatingState' key when constructing the transitions dictionary
                    transitions = {symbol: next_state for symbol, next_state in self.dfa_states[state].items() if symbol != 'isTerminatingState'}
                    transitions_str = str(transitions)
                    if transitions_str not in transitions_partition:
                        transitions_partition[transitions_str] = [state]
                    else:
                        transitions_partition[transitions_str].append(state)

                new_partition.extend(list(transitions_partition.values()))

        # Step 4: Construct minimized DFA
        minimized_dfa_states = {}
        for i, group in enumerate(new_partition):
            state_label = f'Q{i}'
            for state in group:
                minimized_dfa_states[state] = state_label

        # Step 5: Update transitions
        minimized_dfa = {}
        for state, transitions in self.dfa_states.items():
            minimized_state = minimized_dfa_states[state]
            minimized_transitions = {}
            for symbol, next_state in transitions.items():
                if symbol == 'isTerminatingState':
                    minimized_transitions[symbol] = next_state
                else:
                    minimized_transitions[symbol] = minimized_dfa_states[next_state]
            minimized_dfa[minimized_state] = minimized_transitions

        return minimized_dfa
    

    # def visualize(self, name='output/min.gv', view=False):
    #     graph = graphviz.Digraph(engine='dot')
    #     for state, transitions in self.dfa_states.items(): 
    #         if state == 'startingState':
    #             continue
    #         if transitions['isTerminatingState']:
    #             graph.node(state, shape='doublecircle')
    #         else:
    #             graph.node(state, shape='circle')
    #         for char, next_state in transitions.items():
    #             if char == 'isTerminatingState':
    #                 continue
    #             children_states = next_state.split(',')
    #             for child in children_states:
    #                 graph.edge(state, child, label=char)
    #     graph.render("output/min", view=False)


# Usage example:
dfa_states = nfa.nfa2dfa()

minimizer = MinimizeDFA(dfa_states)
# print(minimizer)


states ('S0',)
terminating_states ['S5']
states ('S1',)
terminating_states ['S5']
states ('S2',)
terminating_states ['S5']
states ('S3',)
terminating_states ['S5']
states ('S4',)
terminating_states ['S5']
states ('S5',)
terminating_states ['S5']


In [45]:
import json
from graphviz import Digraph
# importOS
import os

# "C:\Program Files\Graphviz\bin\dot.exe" run this command in the terminal to install graphviz
# ! "C:\Program Files\Graphviz\bin\dot.exe"
os.environ["PATH"] += os.pathsep + 'C:/Program Files/Graphviz/bin/'
# Load state machine definition from JSON file
with open("states.json", "r") as json_file:
    state_machine_definition = json.load(json_file)
#  want to set direction to left to right


# Initialize the Digraph object
dot = Digraph()

# Set the direction of the graph
dot.attr(rankdir="LR")

# Add states to the graph
for state_id, state_info in state_machine_definition["states"].items():
    label = f"{state_id}"
    if state_info['isTerminatingState']:
        dot.node(state_id, label=label, shape="doublecircle")
    else:
        dot.node(state_id, label=label)

# Add transitions to the graph
for state_id, state_info in state_machine_definition["states"].items():
    for event, next_state_id in state_info["Transitions"].items():
        dot.edge(state_id, next_state_id, label=event)

# Save the graph to a file
dot.render("state_machine_graph", format="png", cleanup=True)


'state_machine_graph.png'